In [1]:
from path_arquivos import *
import pandas as pd
caminho = output.var_corte

df_67 = pd.read_excel(caminho, sheet_name= 'CORTE 2025')
df_retorno = pd.read_excel(caminho, sheet_name= "RETORNO")
df_86 = pd.read_excel(ar_xls.ar_86, usecols= ['Código', 'Descrição ', 'Estoque', 'Qtde Pedida', 'Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria', 'Reservado', 'Disponível'])

display(df_67.columns)
display(df_86.columns)

Index(['DATA', 'CARR.', 'COD', 'DESCRIÇÃO', 'PEDIDO', 'QTDE ORIG', 'VL. ORIG',
       'RUA', 'PREDIO', 'APTO', 'ESTOQUE', 'QTDE CORTE', 'VL CORTE', 'HORA',
       'MIN', 'MOTIVO', 'COD FUNC', 'FUNCIONARIO', 'CONCAT', 'COUNT', 'IF',
       'MÊS', 'TURNO'],
      dtype='object')

Index(['Código', 'Descrição ', 'Estoque', 'Qtde Pedida',
       'Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria', 'Reservado',
       'Disponível'],
      dtype='object')

CORPO DO SCRIPTS

In [2]:
df_67 = df_67.loc[df_67['MÊS'] == 'setembro']
df_67['HORAS'] = pd.to_datetime(df_67['HORA'].astype(str) + ":" + df_67['MIN'].astype(str), format='%H:%M').dt.time

var_corte = df_67.groupby('COD').agg(
    QTDE_DIAS = ('DATA', 'nunique'),
    TOTAL_CORTE = ('VL CORTE', 'sum'),
    QTDE_CORTE = ('QTDE CORTE', 'sum')
).reset_index().sort_values(by=['QTDE_DIAS', 'QTDE_CORTE'], ascending= False)
df_86['BLOQUEADO'] = df_86['Bloqueado(Qt.Bloq.-Qt.Avaria)'].fillna(0).astype(int) + df_86['Qt.Avaria'].fillna(0).astype(int)
var_86 = df_86[['Código', 'Descrição ', 'Estoque', 'BLOQUEADO']].copy()
df = var_corte.merge(var_86, left_on= 'COD', right_on= 'Código', how= 'left').drop(columns='Código')
df['var'] = df['BLOQUEADO'] - df['Estoque']

validar = df.loc[(df['var'] < 0) & (df['QTDE_DIAS'] > 2)]
validar = validar.merge(df_retorno, left_on= 'COD', right_on= 'CODPROD', how= "left").drop(columns='CODPROD')
validar[['RETURN', 'DATA']] = validar[['RETURN','DATA']].fillna("")

display(validar)

,COD,QTDE_DIAS,TOTAL_CORTE,QTDE_CORTE,Descrição,Estoque,BLOQUEADO,var,DATA,RETURN
0,444365,4,1875.62,53.0,SUCO NUTRINECTAR PRONTO 200ML UVA,525.0,13.0,-512.0,2025-09-09,Produto na camara-fria
1,464141,3,5224.50,90.0,CREME LEITE TIROL 200GR,254.0,12.0,-242.0,2025-09-10,validade filial
2,471762,3,729.22,22.0,SUCO NUTRINECTAR PRONTO 200ML MARACUJA,2071.0,20.0,-2051.0,NaT,


In [3]:

comp = df_67.loc[df_67['COD'] == 464141]
comp = comp[['DATA','HORAS', 'COD', 'DESCRIÇÃO', 'MOTIVO', 'FUNCIONARIO']].sort_values(by=['DATA','HORAS'], ascending= False)
display(comp)
display(df_retorno)


,DATA,HORAS,COD,DESCRIÇÃO,MOTIVO,FUNCIONARIO
14126,2025-09-05,11:31:00,464141,CREME LEITE TIROL 200GR,MERCADORIA PROXIMO AO VENCIMENTO,CARLOS ALBERTO SANTOS DA SILVA FILHO
14094,2025-09-04,10:36:00,464141,CREME LEITE TIROL 200GR,MERCADORIA PROXIMO AO VENCIMENTO,CARLOS ALBERTO SANTOS DA SILVA FILHO
14027,2025-09-01,09:13:00,464141,CREME LEITE TIROL 200GR,MERCADORIA PROXIMO AO VENCIMENTO,CARLOS ALBERTO SANTOS DA SILVA FILHO


,DATA,CODPROD,RETURN
0,2025-09-09,280000,"inversão, liberado pelo comprador"
1,2025-09-09,444365,Produto na camara-fria
2,2025-09-10,464141,validade filial
3,2025-09-09,468973,3 caixa e 13 unidade no AP
4,2025-09-09,470878,ultima entrada 08/09/2025
5,2025-09-10,450655,ultima entrada 03/09/2025
6,2025-09-10,428511,ultima entrada 03/09/2025
7,2025-09-10,448268,2 unidade no AP
8,2025-09-10,474139,5 unidade no AP
